### 파이선 라이브러리를 활용한 K-Mean 알고리즘 적용 모델 생성 및 적용

- 파이선이 제공하는 다양한 라이브러리
- FIFA 20 선수 데이터를 이용하여 
- K-Mean 알고리즘을 적용하여 선수들을 분류해봅시다!

### 머신러닝 데이터 저장 또는 결과 파일 저장을 위해 사용할 구글 드라이브 접근 권한 데이터

- 코랩 플랫폼을 활용하면 온라인에서 파이선 프로그래밍은 물론 머신러닝, 딥러닝 프로그래밍을 자유롭게 할 수 있다.
- 이 때 자신의 구글 드라이브를 활용하면 AI 학습 데이터 관리나 결과를 파일로 저장하는 데 효과적이다
- 아래에 있는 코드가 바로 이 파일이 자신의 구글 드라이브에 접근할 수 있는 권한을 부여하는 코드이다. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 필요한 라이브러리(도구집)과 데이터 불러오기

- import는 필요한 라이브러리를 불러오고
- 아래 3줄은 Github에 있는 데이터를 불러와 판다스 데이터 프레임 형태로 저장한다.
- 데이터 프레임(DataFrame)이란 행열데이터를 다루기 위한 효과적인 가방(캐리어)라 생각하면 된다!

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

fifa_data = pd.read_csv("https://raw.githubusercontent.com/CarlosQuperman/2021KangwonAIT/main/fifa_player_20.csv")
print(fifa_data.info(verbose=True)) # 중간에 내용이 생략되지 않게 True값을 설정하고 데이터의 형식을 보여줌 
fifa_data.head() #상위 5개값 출력


In [ ]:
# 원본 데이터에서 1000개만 가져오기 (기준은 총능력치 내림차순)

fifa_data = pd.DataFrame.copy(fifa_data.sort_values(by='overall',ascending=).head())

K_features = ['', 'physic', 'dribbling', 'passing', '']
fifa_data_K = pd.DataFrame(fifa_data, columns=K_features) #원본 데이터를 최대한 건들지 말고 새로운 데이터 프레임 생성(엑셀과 비교하여 강점!)
print(fifa_data_K.head())

#NAN은 무엇일까?

                             long_name  physic  dribbling  passing  shooting
0       Lionel Andrés Messi Cuccittini    66.0       96.0     92.0      92.0
1  Cristiano Ronaldo dos Santos Aveiro    78.0       89.0     82.0      93.0
2        Neymar da Silva Santos Junior    58.0       95.0     87.0      85.0
3                            Jan Oblak     NaN        NaN      NaN       NaN
4                          Eden Hazard    66.0       94.0     86.0      83.0


### 공백 데이터 처리 및 각 시각화를 위한 각 열 따로 저장

In [ ]:
fifa_data_K = fifa_data_K.dropna(axis=0) #공백 데이터 처리
print(fifa_data_K.head())

Name  = fifa_data_K['long_name']
Physic = fifa_data_K['physic']
Dribbling= fifa_data_K['dribbling']
Passing = fifa_data_K['passing']
Shooting = fifa_data_K['shooting']


                             long_name  physic  dribbling  passing  shooting
0       Lionel Andrés Messi Cuccittini    66.0       96.0     92.0      92.0
1  Cristiano Ronaldo dos Santos Aveiro    78.0       89.0     82.0      93.0
2        Neymar da Silva Santos Junior    58.0       95.0     87.0      85.0
4                          Eden Hazard    66.0       94.0     86.0      83.0
5                      Kevin De Bruyne    78.0       86.0     92.0      86.0


### 선수 능력 분포 산점도로 시각화(신체능력, 드리블)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Fifa Player Info")
plt.scatter(Physic,Dribbling,c='blue',marker='o',label='Player')
plt.xlabel('Physics')
plt.ylabel('Dribbling')
plt.grid()
plt.legend(loc='best')
plt.savefig('')
plt.show()


### K-Mean 알고리즘 적용하기!

* K-Mean 알고리즘을 적용하여 선수들을 3개의 군집으로 분류해보자.
* 결과는 DataFrame result에 저장하고
* 분류된 결과를 csv 파일로 저장해보자.

In [ ]:
label_features = ['physic', 'dribbling'] #군집의 기준으로 사용될 선수 능력 
label_data_K = pd.DataFrame(fifa_data_K, columns=label_features) #신체능력과 드리블 능력을 가진 새로운 데이터 프레임 생성 
print(label_data_K.info()) #생성된 데이터 프레임 정보 확인 

fifa_model = KMeans(n_clusters=3,algorithm='auto') #모델 정의 
fifa_model.fit(label_data_K) #모델 훈련(알고리즘 적용)

result = label_data_K.copy() #결과 저장을 위해 제일 위 3줄에서 생성된 데이터 프레임 복사
result['cluster'] = fifa_model.labels_ #result에 새로운 열 cluster 추가 (K-Mean 알고리즘을 적용해서 나온 군집 결과)
result['Full Name'] = Name #이름 확인을 위해 새로운 열 Full Name 추가
result.head() #5개의 값 확인 
#result.tail()
result.to_csv('',index=,encoding='') #결과값을 파일로 저장

### 분류 결과 시각화 및 저장 1(Matplotlib)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Fifa Player Label Info")
plt.scatter(result[''],result[''],c=result['cluster'],alpha=0.5,marker='o',label='Player') #c는 색깔, alpha는 투명도
plt.xlabel('Physic')
plt.ylabel('Dribbling')
plt.grid( )
plt.legend(loc='best')
plt.savefig('')
plt.show()


### 분류 결과 시각화 및 저장2 (Seaborn)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Fifa Player Label Info")
sns.scatterplot(x=result['physic'],y=result['dribbling'],hue='cluster',data=result,palette='Set2')
plt.xlabel('Physic')
plt.ylabel('Dribbling')
plt.legend(loc='best')
plt.savefig('/content/gdrive/MyDrive/CSV/K-mean 분포2(sns).png')
plt.show()

### 위에 있는 내용을 이해하고 활용하기

- 위 내용들을 잘 이해하고 복사 후 수정하면
- 신체적 능력과 드리블 능력 이외의 기준으로도 K-Mean 알고리즘을 적용하여 결과를 확인할 수 있다.
- 또 미리 준비한 데이터뿐 아니라 내가 준비한 데이터를 드라이브에 업로드하고 이를 데이터프레임으로 불러올 수 있다.